In [1]:
import pandas as pd

In [2]:
# Load the uploaded CSV file
file_path = "./before_preprocessed_inventory_data.csv"
df = pd.read_csv(file_path)

# Display the first few rows to understand the structure
df.head()

,Date,Store_ID,Product_ID,Category,Region,Inventory_Level,Units_Sold,Units_Ordered,Demand_Forecast,Price,Discount,Weather_Condition,Holiday/Promotion,Competitor_Pricing,Seasonality,Lead_Time
0,1/1/2022,S001,P0001,Groceries,North,231,2,55,135.47,33.50,20,Rainy,0,29.69,Autumn,9
1,1/1/2022,S001,P0002,Toys,South,204,4,66,144.04,63.01,20,Sunny,0,66.16,Autumn,22
2,1/1/2022,S001,P0003,Toys,West,102,1,51,74.02,27.99,10,Sunny,1,31.32,Summer,22
3,1/1/2022,S001,P0004,Toys,North,469,0,164,62.18,32.72,10,Cloudy,1,34.74,Autumn,22
4,1/1/2022,S001,P0005,Electronics,East,166,5,135,9.26,73.64,0,Sunny,0,68.95,Summer,17


In [3]:
# Convert 'Date' to datetime format and extract 'month/year'
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Month'] = df['Date'].dt.to_period('M').astype(str)

# Group by month and product ID to compute sales speed and stock level at month's end
grouped = df.sort_values('Date').groupby(['Month', 'Product_ID'])

# Calculate average monthly units sold
sales_speed = grouped['Units_Sold'].sum().rename("Sales_Speed")

# Get last inventory level of the month
stock_level = grouped['Inventory_Level'].last().rename("Stock_Level")

# Get the first occurrence of product metadata (Category, Lead_Time, Price)
meta = grouped[['Category', 'Lead_Time', 'Price']].first()

# Combine all into a single DataFrame
preprocessed_df = pd.concat([sales_speed, stock_level, meta], axis=1).reset_index()

# Save to CSV
output_path = "./preprocessed_inventory_data.csv"
preprocessed_df.to_csv(output_path, index=False)

preprocessed_df.head()

,Month,Product_ID,Sales_Speed,Stock_Level,Category,Lead_Time,Price
0,2022-01,P0001,370,311,Groceries,9,33.50
1,2022-01,P0002,368,290,Clothing,10,41.13
2,2022-01,P0003,383,55,Clothing,10,89.32
3,2022-01,P0004,380,230,Groceries,9,87.23
4,2022-01,P0005,378,338,Groceries,9,81.72
